In [113]:
import pandas as pd
import re
import us

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)


In [130]:
df = pd.read_csv("posts.csv",index_col=0, delimiter=";", error_bad_lines=False)

In [131]:

df = df[["post"]]
df["post"] = df["post"].str.strip("gt.classof2025")
df["post"] = df["post"].str.replace("’","'")
df


,post
2021-06-24T17:41:34.000Z,"I'm Diego from West Palm Beach, FL and I plan ..."
2021-06-24T17:40:16.000Z,"Hey guys, my name's Avanish and I'm from the n..."
2021-06-24T17:34:58.000Z,hey guys!! my name is marie and I'm committed ...
2021-06-23T19:37:58.000Z,"Hi! My name is Megan and I'm from Savannah, Ga..."
2021-06-23T16:42:49.000Z,"hey guys!! my name's ali, and ive been committ..."
...,...
2020-12-21T22:55:08.000Z,"Hello everyone! My name is Claudia Vitale, and..."
2020-12-21T21:35:08.000Z,Hey guys:))) My name is Salma and I'm from Gwi...
2020-12-20T17:48:02.000Z,Hey everyone! My name is Jen O'Brien and I'm ...
2020-12-19T21:22:54.000Z,hey guys! i'm mackenzie and i'm from marietta ...


In [132]:
def get_name(post):
    name = re.search(r"[mM]y [Nn]ame'?s ([A-z]* ?[A-z]*)",post)
    if name:
        return name.group(1).replace(" and", "").replace(" from", "")
    name = re.search(r"[mM]y [Nn]ame is ([A-z]* ?[A-z]*)",post)
    if name:
        return name.group(1).replace(" and", "").replace(" from", "")
    name = re.search(r"[Ii][' a]*m ([A-z]* ?[A-z]*)[ ,\.!]", post)
    if name:
        return name.group(1).replace(" and","").replace(" from", "")

def get_hometown(post, state=False):
    hometown = re.search(r"from ([A-z]*[ ,]*(([A-z]{2}[ ,.])|[A-z ]*))",post)
    if hometown:
        if state:
            return hometown.group(3)
        splitters = [" and",' but',' I ',' i ']
        hometown = hometown.group(1).strip(" ,")
        for splitter in splitters:
            hometown = hometown.split(splitter)[0]
        return hometown
    hometown = re.search(r"in ([A-z]*[ ,]*(([A-z]{2}[ ,.])|[A-z ]*))",post)
    if hometown:
        if state:
            return hometown.group(3)
        splitters = [" and",' but',' I ',' i ']
        hometown = hometown.group(1).strip(" ,")
        for splitter in splitters:
            hometown = hometown.split(splitter)[0]
        return hometown

def get_state(hometown, post):
#     print(hometown, post)

    def contains_word(text, word):
        return bool(re.search(r'\b' + re.escape(word) + r'\b', text))

    containsHT = True
    state = None
    if not hometown:
        containsHT = False
    if containsHT:
        if re.search(r"[Aa]tlanta", hometown):
            return "GA"
        if re.search(r"[Gg]eorgia", hometown):
            return "GA"
        
    if state == None:
        for s in us.states.STATES:
            if contains_word(post, s.name) or contains_word(post, s.name.lower()) or contains_word(post,s.abbr):
                state = s.abbr
        if "DC" in post or "D.C." in post:
            state =  "DC"
    return state

def get_region(state, post):
    regions = {
        "georgia": ["GA"],
        "midwest": ["ND","SD","NB","KS", "MN","MO","IA","WI","IL","IN","MI", "OH"],
        "rockies": ["MT","ID","WY","NV","UT","CO"],
        "southwest": ["AZ","NM","TX","OK", "AR"],
        "pacific": ["WA","OR","CA"],
        "southeast": ["LA","MS","AL","SC","NC","VA","WV","KY","TN","FL"],
        "northeast": ["PA","NY","NJ","CT","RI","MA","VT","NH","ME", "DC","DE","MD"]
    }
    for r, states in regions.items():
        if state in states:
            return r
            

def get_major(post):
    major = re.search(r"major(?:ing)? in ([A-z]* [A-z]*)[ ,.]", post)
    if major:
        return major.group(1).strip(" and")
    major = re.search(r"([A-z]* [A-z]*) major", post)
    if major:
        return major.group(1).strip(" and")

def get_insta(post):
    insta = re.search(r"i?n?s?t?a?g?r?a?m?: @?([0-z.]*)",post, re.IGNORECASE)
    if insta:
        return insta.group(1).replace("sc:","").replace("Snap:","").replace("snap:","")

df["name"] = df["post"].apply(get_name)
df["hometown"] = df["post"].apply(get_hometown)
df["region"] = ["" for i in range(len(df))] 
df["state"] = ["" for i in range(len(df))] 
for index, elem in df.iterrows():
    df.loc[index, "state"] = get_state(elem.hometown, elem.post)
    df.loc[index, "region"] = get_region(elem.state, elem.post)
df["major"] = df["post"].apply(get_major)
df['insta'] = df['post'].apply(get_insta)
df["gender"] = ["" for i in range(len(df))]

df

,post,name,hometown,region,state,major,insta,gender
2021-06-24T17:41:34.000Z,"I'm Diego from West Palm Beach, FL and I plan ...",Diego,West Palm Beach,southeast,FL,Industrial Engineering,diegom0ra,
2021-06-24T17:40:16.000Z,"Hey guys, my name's Avanish and I'm from the n...",Avanish,the northwest Chicago suburbs,georgia,GA,electrical engineering,avanish_n_Sc:,
2021-06-24T17:34:58.000Z,hey guys!! my name is marie and I'm committed ...,marie,Environmental Science,None,None,with,None,
2021-06-23T19:37:58.000Z,"Hi! My name is Megan and I'm from Savannah, Ga...",Megan,"Savannah, Ga.",None,None,International Affairs,None,
2021-06-23T16:42:49.000Z,"hey guys!! my name's ali, and ive been committ...",ali,"Evans, Georgia for the past",georgia,GA,bme,None,
...,...,...,...,...,...,...,...,...
2020-12-21T22:55:08.000Z,"Hello everyone! My name is Claudia Vitale, and...",Claudia Vitale,"robotics,",None,None,Mechanical engineering,None,
2020-12-21T21:35:08.000Z,Hey guys:))) My name is Salma and I'm from Gwi...,Salma,Gwinnett county,None,None,biochemistry,_sg.xox,
2020-12-20T17:48:02.000Z,Hey everyone! My name is Jen O'Brien and I'm ...,Jen O,engineering,None,None,Industrial engineering,None,
2020-12-19T21:22:54.000Z,hey guys! i'm mackenzie and i'm from marietta ...,mackenzie,marietta,None,None,biomedical engineering,None,


In [133]:

boys = open('boynames.txt', 'r')
boys = boys.read().split('\n')

girls = open('girlnames.txt', 'r')
girls = girls.read().split('\n')

def add_boy(name):
    f = open('boynames.txt','a')
    f.write(f"\n{name}")
    f.close()

def add_girl(name):
    f = open('girlnames.txt','a')
    f.write(f"\n{name}")
    f.close()

In [134]:
for i in range(len(df)):
    if not df["name"].iloc[i]:
        continue
    name = df["name"].iloc[i].split(' ')[0]
    name = name[0].upper() + name[1:].lower()
    if name in boys:
        df["gender"].iloc[i] = "B"
    elif name in girls:
        df["gender"].iloc[i] = "G"
    else:
        df["gender"].iloc[i] = "BLANK"

In [135]:



for i in range(len(df)):
    if not df["name"].iloc[i]:
        continue
    name = df["name"].iloc[i].split(' ')[0]
    name = name[0].upper() + name[1:].lower()
    if name in boys:
        df["gender"].iloc[i] = "B"
        continue
    elif name in girls:
        df["gender"].iloc[i] = "G"
        continue
    inp = input(f"{name}?(B/b/q/S/g/G) ")
    if not inp:
        continue
    inp = inp[0]
    if inp == "B":
        df["gender"].iloc[i] = "B"
        add_boy(name)
        boys.append(name)
    elif inp == "b":
        df["gender"].iloc[i] = "B"
    elif inp == 'g':
        df["gender"].iloc[i] = "G"
    elif inp == 'G':
        df["gender"].iloc[i] = "G"
        add_girl(name)
        girls.append(name)
    elif inp == "S":
        continue
    elif inp in ['q','Q']:
        break

Avanish?(B/b/q/S/g/G) S
Marie?(B/b/q/S/g/G) S
Ali?(B/b/q/S/g/G) S
Shasha?(B/b/q/S/g/G) g
Shivom?(B/b/q/S/g/G) S
Xinyi?(B/b/q/S/g/G) S
Freddie?(B/b/q/S/g/G) B
Emir?(B/b/q/S/g/G) S
Sami?(B/b/q/S/g/G) S
Kem?(B/b/q/S/g/G) S
Sarvesh?(B/b/q/S/g/G) S
Sterling?(B/b/q/S/g/G) S
Aminatou?(B/b/q/S/g/G) S
Marc?(B/b/q/S/g/G) B
Shashank?(B/b/q/S/g/G) S
Edo?(B/b/q/S/g/G) S
Teodora?(B/b/q/S/g/G) S
Karam?(B/b/q/S/g/G) S
Jadyn?(B/b/q/S/g/G) S
Azeh?(B/b/q/S/g/G) S
Jiwhan?(B/b/q/S/g/G) S
Lucia?(B/b/q/S/g/G) G
Malen?(B/b/q/S/g/G) S
Will?(B/b/q/S/g/G) B
Bennett?(B/b/q/S/g/G) B
Shirley?(B/b/q/S/g/G) G
Aditya?(B/b/q/S/g/G) S
Krish?(B/b/q/S/g/G) S
Walker?(B/b/q/S/g/G) B
Marta?(B/b/q/S/g/G) q


In [136]:

df = df[df["gender"] != "G"]

df[["name","hometown","state","region","gender","major",'insta','post']].to_csv("new.csv")